In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
%cd drive/My\ Drive/seq2seq_3/

Mounted at /content/drive/
/content/drive/My Drive/seq2seq_3


In [ ]:
!pip install biopython
!pip install tensorflow==2.5.0
#%tensorflow_version 2.5.0
%matplotlib notebook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from utils.prepare_data import preproccess_data
from utils.deep_model.s2s import s2s_model, dump, load_pkl
from utils.deep_model.deep_s2s import deep_s2s_model
from utils.deep_model.attn_deep_s2s import attn_deep_s2s_model
from utils.evaluation import evaluate_s2s
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: ignored

In [ ]:
experiments = {'Deep_BI_GRU':deep_s2s_model} #{ 'LSTM':s2s_model, 'GRU':s2s_model} 
                
#{'Deep_BI_GRU':deep_s2s_model, 'Deep_LSTM':deep_s2s_model, 'Deep_BI_LSTM':deep_s2s_model, 'Deep_GRU':deep_s2s_model }
 #{'Attn_Deep_BI_LSTM':attn_deep_s2s_model, 'Attn_Deep_BI_GRU':attn_deep_s2s_model}
for key, value in experiments.items():
  #print(key, '', value)
  model_name = key
  model = value
  embed_size = 50
  test_size = .2
  epochs = 500
  batch_size = 128
  val_split = .5
  patience = 11
  num_hidden_cells = 32
  lr = 0.001
  decay_step_size = 50
  decay_factor = 0.98
  dropout = 0.2
  preproc = preproccess_data( flu_csv_file = 'prepared_data/flu_data_x2.csv',
                              no_of_char_per_word = 1,
                              seq_cols_list = ['virusA_seq', 'virusB_seq'],
                              input_col = 'virusB_seq',
                              target_col = 'virusA_seq',
                              antigenic_distance_col = 'Antigenic_Distance',
                              embedding_size = embed_size,
                              add_begin_end = True,
                              randomize_df = False,
                              antigenic_distance_scale = 'standard',
                              test_set_size = test_size

                            )
  preproc.preproc_data()

  s2s_model = model( exp_title = model_name,
                        loss='categorical_crossentropy',
                        metrics=['acc'],
                        recurrent_layer = model_name,
                        embedding_size = embed_size,
                        num_encoder_tokens = preproc.input_num_tokens,
                        num_hidden_cells = num_hidden_cells,
                        num_decoder_tokens=preproc.target_num_tokens,
                        dropout = dropout,
                        optimizer = 'adam',
                        lr = lr,
                        max_target_seq_len = preproc.target_max_seq_len, # not used
                        decay_step_size = decay_step_size,
                        decay_factor = decay_factor
                        )
  s2s_model.build()
  s2s_model.model.summary()

  import tensorflow as tf
  from tensorflow.python.profiler.model_analyzer import profile
  from tensorflow.python.profiler.option_builder import ProfileOptionBuilder
  print('TensorFlow:', tf.__version__)
  model= s2s_model.model

  input_signature = [(tf.TensorSpec(shape=(1,375) ), 
    tf.TensorSpec(shape=(1,100)),
    tf.TensorSpec(shape=(1,50)),
    tf.TensorSpec(shape=(1,None,100)),
    )]
  forward_pass = tf.function(
    model.call,
    input_signature=input_signature
    )

  graph_info = profile(forward_pass.get_concrete_function().graph,
                        options=ProfileOptionBuilder.float_operation())

  # The //2 is necessary since `profile` counts multiply and accumulate
  # as two flops, here we report the total number of multiply accumulate ops
  flops = graph_info.total_float_ops // 2
  print('Flops: {:,}'.format(flops))
  s2s_model.inf_encoder_model()
  s2s_model.inf_decoder_model()
  s2s_model.train(preproc.encoder_input_data_train, preproc.decoder_input_data_train, preproc.enc_antigenic_distance_train, preproc.decoder_target_data_train, epochs, batch_size, val_split,patience)
  
  plt.plot(s2s_model.history.history['acc'], color = 'green')
  plt.plot(s2s_model.history.history['val_acc'], color = 'red')
  plt.title(s2s_model.recurrent_layer + ' Model Accuracy')
  plt.yticks([.1, .2, .3, .4, .5, .6, .7, .8, .9])
  plt.ylabel('Accuracy')
  plt.xlabel('Epochs')
  plt.legend(['Train', 'Validation'], loc='best')
  plt.savefig(s2s_model.exp_id + '/' + s2s_model.recurrent_layer + '_model_accuracy.pdf')
  plt.show()
  plt.clf()
  plt.cla()
  plt.close()

  plt.plot(s2s_model.history.history['loss'], color = 'blue')
  plt.plot(s2s_model.history.history['val_loss'], color = 'red')
  plt.title(s2s_model.recurrent_layer + ' Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epochs')
  plt.legend(['Train', 'Validation'], loc='best')
  plt.savefig(s2s_model.exp_id + '/' +  s2s_model.recurrent_layer + '_model_loss.pdf')
  plt.show()
  plt.clf()
  plt.cla()
  plt.close()

  model_eval = evaluate_s2s(
                            test_index = preproc.test_index,                                                                       
                            max_len_decoded_sentence = 11+327*2,
                            no_recorde_to_evaluate = len(preproc.test_index),
                            encoder_input_data_test = preproc.encoder_input_data_test,
                            enc_antigenic_distance_test = preproc.enc_antigenic_distance_test,
                            target_token_index = preproc.target_token_index,
                            reverse_target_index = preproc.target_reverse_index,
                            seq_model = s2s_model,
                            results_df_file_name = 'results',
                            summary_df_file_name = 'summary',
                            real_virus_name_df_col = preproc.flu_df.virusA,
                            real_vireus_df_col = preproc.flu_df.virusA_seq
                          )
  model_eval.evaluate_model()
  np.save(s2s_model.exp_id + '/' + s2s_model.recurrent_layer + 'history.npy',s2s_model.history.history)
  

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     1150        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
tf.__operators__.add (TFOpLambd (None, None, 50)     0           embedding[0][0]                  
                                                                 input_2[0][0]                

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:

print(s2s_model.encoder_inputs.shape)
print(s2s_model.encoder_model.input_shape)
print(s2s_model.decoder_model.input_shape)
print(s2s_model.encoder_inputs.shape)
print(s2s_model.decoder_inputs.shape)
print(s2s_model.enc_ad_input.shape)
print(s2s_model.decoder_outputs)
#self.encoder_inputs, self.decoder_inputs, self.enc_ad_input], decoder_outputs

(None, None)
[(None, None), (None, None)]
[(None, None), (None, 64)]
(None, None)
(None, None)
(None, None)
(None,)
